## Embedding the Query

In [ ]:
import numpy as np

from fastembed import TextEmbedding

In [2]:
query = "I just discovered the course. Can I join now?"

In [3]:
embedding = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

In [30]:
emb_query, = list(embedding.embed(query))

In [31]:
len(emb_query)

512

In [37]:
emb_query.min()

np.float64(-0.11726373885183883)

In [ ]:
np.linalg.norm(emb_query)

np.float64(1.0)

## Cosine Similarity

In [38]:
doc = "Can I still join the course after the start date?"

In [39]:
emb_doc, = list(embedding.embed(doc))

In [40]:
emb_query.dot(emb_doc)

np.float64(0.9008528895674548)

## Ranking by Cosine

In [41]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
emb_docs = []

for doc in documents:
    embedded_doc, = list(embedding.embed(doc['text']))
    emb_docs.append(embedded_doc)

emb_docs_matrix = np.array(emb_docs)

In [49]:
emb_docs_matrix.dot(emb_query)

array([0.76296847, 0.81823782, 0.80853974, 0.7133079 , 0.73044992])

### Question 4

In [56]:
emb_full_text = []

for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    embedded_full_text, = embedding.embed(full_text)
    emb_full_text.append(embedded_full_text)


emb_full_text_matrix = np.array(emb_full_text)

In [57]:
emb_full_text_matrix.dot(emb_query)

array([0.85145432, 0.84365942, 0.8408287 , 0.7755158 , 0.80860078])

## Selecting Embedding Model

In [58]:
import json

for model in TextEmbedding.list_supported_models():
    if model["dim"] <= 512:
        print(f"Dimension:", model["dim"])
        print(json.dumps(model, indent=2))

Dimension: 384
{
  "model": "BAAI/bge-small-en",
  "sources": {
    "hf": "Qdrant/bge-small-en",
    "url": "https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.13,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
Dimension: 384
{
  "model": "BAAI/bge-small-en-v1.5",
  "sources": {
    "hf": "qdrant/bge-small-en-v1.5-onnx-q",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.067,
  "additional_files": [],
  "dim": 384,
  "tasks": {}
}
Dimension: 512
{
  "model": 

## Indexing Qdrant

In [59]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [60]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")

In [68]:
model_handle = "BAAI/bge-small-en"
collection_name="homework_2"
embedding_size = 384

In [ ]:
from qdrant_client import models

# Create the collection with specified sparse vector parameters
client.create_collection(
    collection_name=collection_name
    vectors_config=models.VectorParams(
        size = embedding_size,
        distance = models.Distance.COSINE
    )
)

True

In [67]:
points = []

for i,doc in enumerate(documents):

    text = doc['question'] + ' ' + doc['text'] 
    vector = models.Document(text=text, model=model_handle)

    point = models.PointStruct(
        id=i,
        vector=models.Document(text=doc['text'], model=model_handle), 
        payload=doc
    )
    
    points.append(point)

In [69]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

c:\Users\FrancescaBellucci\OneDrive - WCC Development B.V\Bureaublad\Francesca\Projects\LLM ZoomCamp\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\FrancescaBellucci\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json: 0.00B [00:00, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [74]:
def vector_search(query):

    vector_results = client.query_points(
        collection_name=collection_name,
        query=models.Document( 
            text=query,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value="machine-learning-zoomcamp")
                )
            ]
        ),
        limit=5, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return vector_results.points

In [75]:
vector_search(query)

[ScoredPoint(id=62, version=0, score=0.8661978, payload={'text': 'Yes, you can. We will also do that next week, so don’t worry, you will learn how to do it.', 'section': '2. Machine Learning for Regression', 'question': 'Can I use LinearRegression from Scikit-Learn for this week?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=16, version=0, score=0.86261594, payload={'text': 'No, it’s not possible. The form is closed after the due date. But don’t worry, homework is not mandatory for finishing the course.', 'section': 'General course-related questions', 'question': 'Can I submit the homework after the due date?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=372, version=0, score=0.8562715, payload={'text': 'Yes, it’s allowed (as per Alexey).\nAdded By Rileen Sinha', 'section': 'Miscellaneous', 'question': 'Is use of libraries like fast.ai or huggingface allowed in the ca